In [1]:
import pickle; import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import signal 
import scipy
import h5py
import tqdm

/home/adrian/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sf = pickle.load(open("../data/SF.pickle", "rb"))
#0-217 blazar, 218-793 CV
DATA_SOURCE = "../data/images.h5"
NUM_BINS = 100

def make_one_picture(idx):
    (timediff, magdiff) = (sf[idx]['timediff'], sf[idx]['magdiff'])
    # scale the data to be 3000 x 3
    def scale_to_square(_diff):
        _diff = _diff - np.min(_diff)
        _diff = _diff * 3.0 / np.max(_diff)
        return _diff
    scaled_tdiff = scale_to_square(timediff)
    scaled_mdiff = scale_to_square(magdiff)
    
    H, xedges, yedges = np.histogram2d(scaled_tdiff, scaled_mdiff, bins=NUM_BINS)
    b, a = signal.butter(10, 0.05)
    H = signal.lfilter(b, a, H)
    if sf[idx]['class'] == 'Blazar':
        c = 0
    else:
        c = 1
    return np.array(H), c

def plot_picture(idx):
    H = make_one_picture()
    fig = plt.figure(figsize=(30, 30))
    plt.imshow(H, interpolation='nearest', origin='low')
    plt.colorbar()
    plt.savefig('../images/%s-%d.png' % (sf[idx]['class'], idx))
    
# X = []
# y = []
# for i in tqdm.tqdm(range(len(sf))):
#     _x, _y = make_one_picture(i)
#     X.append(_x); y.append(_y)
    
# X = np.array(X)
# y = np.array(y)
    
# with h5py.File(DATA_SOURCE) as f:
#     f["X"] = X
#     f['y'] = y

In [3]:
with h5py.File(DATA_SOURCE) as f:
    X = f['X'][:]
    y = f['y'][:]
    
X = np.reshape(X, (-1,NUM_BINS,NUM_BINS,1))

KeyError: "Unable to open object (object 'X' doesn't exist)"

In [4]:
from keras import optimizers
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Using TensorFlow backend.


NameError: name 'X' is not defined

In [5]:
def CNN():
    from keras.models import Sequential
    from keras.layers import Conv2D, Activation, Dense, BatchNormalization, Dropout
    from keras.activations import relu
    from keras.layers.core import Flatten
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(5, 5), strides=(1,1), padding='same', input_shape=(NUM_BINS,NUM_BINS,1)))
    model.add(Activation('relu'))
#     model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Conv2D(64, kernel_size=(5, 5), strides=(1,1), padding='same', input_shape=(NUM_BINS,NUM_BINS,64)))
    model.add(Activation('relu')) 
#     model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Conv2D(10, kernel_size=(5, 5), strides=(1,1), padding='same', input_shape=(NUM_BINS,NUM_BINS,64)))
    model.add(Activation('relu')) 
#     model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [6]:
print(X_train.shape, y.shape)
num_epochs = 30
cnn = CNN()
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001)
cnn.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
for epoch in range(num_epochs):
    print("Epoch is", epoch)
    cnn.fit(X_train, y_train)

pred = np.reshape(cnn.predict(X_test), len(pred))
acc = float(np.count_nonzero(np.equal(np.round(pred), y_test))) / len(pred)
print(acc)

NameError: name 'X_train' is not defined